In [1]:
#Import/install everything needed
!pip install folium
import requests
import numpy as np
#pandas will be used for dataframes and for wikipedia scrapping
import pandas as pd 
from pandas.io.json import json_normalize
#geocode library is need for longitudes and latitudes
import geopy.geocoders
#Folium for visualization
import folium
#KMeans needed for clustering
from sklearn.cluster import KMeans

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#pandas .read_html will be used to scrape the table. First table in the page is indexed as 0
df = pd.read_html(url, attrs={"class": "wikitable sortable"})[0] 
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [3]:
#not assigned neighboroughs can be removed simply using inequality sign
df=df[df.Borough != "Not assigned"]
df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [4]:
df.shape

(103, 3)

In [5]:
# Arc.GIS is a simple a ready to use feature part of geopy
nom=geopy.geocoders.ArcGIS()

In [6]:
#To get geocodes a full address is needed including Toronto together with the postal code
df["full_address"]=df["Postal Code"]+", Toronto"
df["geocodes"]=df["full_address"].apply(nom.geocode)
df["Latitude"]=df["geocodes"].apply(lambda x: x.latitude)
df["Longitude"]=df["geocodes"].apply(lambda x: x.longitude)

In [7]:
df.head()

,Postal Code,Borough,Neighbourhood,full_address,geocodes,Latitude,Longitude
2,M3A,North York,Parkwoods,"M3A, Toronto","(M3A, (43.75245000000007, -79.32990999999998))",43.75245,-79.32991
3,M4A,North York,Victoria Village,"M4A, Toronto","(M4A, (43.73057000000006, -79.31305999999995))",43.73057,-79.31306
4,M5A,Downtown Toronto,"Regent Park, Harbourfront","M5A, Toronto","(M5A, (43.65512000000007, -79.36263999999994))",43.65512,-79.36264
5,M6A,North York,"Lawrence Manor, Lawrence Heights","M6A, Toronto","(M6A, (43.72327000000007, -79.45041999999995))",43.72327,-79.45042
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government","M7A, Toronto","(M7A, (43.66253000000006, -79.39187999999996))",43.66253,-79.39188


In [8]:
df=df.reset_index(drop=True)
df.head()

,Postal Code,Borough,Neighbourhood,full_address,geocodes,Latitude,Longitude
0,M3A,North York,Parkwoods,"M3A, Toronto","(M3A, (43.75245000000007, -79.32990999999998))",43.75245,-79.32991
1,M4A,North York,Victoria Village,"M4A, Toronto","(M4A, (43.73057000000006, -79.31305999999995))",43.73057,-79.31306
2,M5A,Downtown Toronto,"Regent Park, Harbourfront","M5A, Toronto","(M5A, (43.65512000000007, -79.36263999999994))",43.65512,-79.36264
3,M6A,North York,"Lawrence Manor, Lawrence Heights","M6A, Toronto","(M6A, (43.72327000000007, -79.45041999999995))",43.72327,-79.45042
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government","M7A, Toronto","(M7A, (43.66253000000006, -79.39187999999996))",43.66253,-79.39188


In [9]:
#Have to drop 2 columns
df.drop(columns=['full_address','geocodes'], inplace=True)
df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.75245,-79.32991
1,M4A,North York,Victoria Village,43.73057,-79.31306
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66253,-79.39188


In [10]:
#General map of Toronto
Toronto=nom.geocode("Toronto, Ontario")[1]
toronto_map = folium.Map(location=[Toronto[0],Toronto[1]], zoom_start=10)

#Markers needed on the map. The zip function consent to iterate multiple items to pass every combination
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        popup=label,
        radius=3,
        color='red',
        parse_html=False).add_to(toronto_map)
    
display(toronto_map)  

In [11]:
#Foursquare Credentials
CLIENT_ID = 'TSSEO2GLYRF2QAVCD3BLFFX2E0N4DLBSJBHX5TIVKQCOK51L'
CLIENT_SECRET = 'JVRZW2HRILQKO1QXHSO4R2OSWQBA2CEYXYKWV0S1FTKFRZ1S'
ACCESS_TOKEN = 'U42HD0YQJITMN0CXV15NUN2DUB5FMBLGFPWUXLD5IFZZS2HJ' 
VERSION = '20201231'
LIMIT = 1000

In [12]:
#I'm going to perform an analysis similar to the one performed in the previous lab.
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [13]:
#Define a function to get venues for each neighbourhood
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [14]:
toronto_venues = getNearbyVenues(names=df['Neighbourhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [15]:
#Little check
print(toronto_venues.shape)
toronto_venues.head()

(2377, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.75245,-79.32991,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.75245,-79.32991,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.73057,-79.31306,Wigmore Park,43.731023,-79.310771,Park
3,Victoria Village,43.73057,-79.31306,Memories of Africa,43.726602,-79.312427,Grocery Store
4,"Regent Park, Harbourfront",43.65512,-79.36264,Roselle Desserts,43.653447,-79.362017,Bakery


In [16]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column


neigh_names = toronto_onehot['Neighborhood']
toronto_onehot.drop(labels=['Neighborhood'], axis=1,inplace = True)
toronto_onehot.insert(0, 'Neighborhood', neigh_names)
toronto_onehot.head()

,Neighborhood,ATM,Accessories Store,Adult Boutique,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,...,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
 #Group rows by Neighborhood
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,ATM,Accessories Store,Adult Boutique,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,...,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0000,0.083333,0.000000,0.0,0.000000,0.000000,0.000000
1,"Alderwood, Long Branch",0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
3,Bayview Village,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
4,"Bedford Park, Lawrence Manor East",0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
5,Berczy Park,0.0,0.000000,0.000000,0.000000,0.016393,0.0,0.016393,0.000000,0.000000,...,0.016393,0.000000,0.000000,0.0000,0.000000,0.000000,0.0,0.000000,0.000000,0.016393
6,"Birch Cliff, Cliffside West",0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
7,"Brockton, Parkdale Village, Exhibition Place",0.0,0.011905,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.023810,...,0.011905,0.000000,0.000000,0.0000,0.000000,0.000000,0.0,0.000000,0.000000,0.011905
8,"Business reply mail Processing Centre, South C...",0.0,0.000000,0.000000,0.020000,0.000000,0.0,0.010000,0.000000,0.000000,...,0.020000,0.000000,0.000000,0.0000,0.000000,0.010000,0.0,0.000000,0.000000,0.000000
9,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.012821,0.000000,0.0000,0.000000,0.000000,0.0,0.000000,0.000000,0.012821


In [18]:
#Print each Neighborhood for the top 5 most common venues
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                venue  freq
0  Chinese Restaurant  0.17
1         Supermarket  0.08
2      Discount Store  0.08
3       Grocery Store  0.08
4              Bakery  0.08


----Alderwood, Long Branch----
                   venue  freq
0      Convenience Store  0.25
1                    Pub  0.25
2                    Gym  0.25
3  Performing Arts Venue  0.25
4      Mobile Phone Shop  0.00


----Bathurst Manor, Wilson Heights, Downsview North----
                       venue  freq
0           Business Service   1.0
1                        ATM   0.0
2                  Nightclub   0.0
3  Middle Eastern Restaurant   0.0
4         Miscellaneous Shop   0.0


----Bayview Village----
                        venue  freq
0                       Trail  0.50
1  Construction & Landscaping  0.25
2                        Park  0.25
3                   Nightclub  0.00
4     New American Restaurant  0.00


----Bedford Park, Lawrence Manor East----
                     venue  freq
0       

In [19]:
#Put into a pandas DF, define the function
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [20]:
#Create the df
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Chinese Restaurant,Bakery,Grocery Store,Shopping Mall,Supermarket,Sushi Restaurant,Badminton Court,Discount Store,Newsagent,Bubble Tea Shop
1,"Alderwood, Long Branch",Gym,Convenience Store,Performing Arts Venue,Pub,Falafel Restaurant,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant
2,"Bathurst Manor, Wilson Heights, Downsview North",Business Service,Yoga Studio,Dry Cleaner,Flea Market,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm
3,Bayview Village,Trail,Construction & Landscaping,Park,Yoga Studio,Falafel Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space
4,"Bedford Park, Lawrence Manor East",Sandwich Place,Italian Restaurant,Coffee Shop,Japanese Restaurant,Juice Bar,Restaurant,Pub,Thai Restaurant,Indian Restaurant,Sports Club


In [21]:
#Cluster Neighborhoods
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
# set number of clusters
kclusters = 5
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 3, 0, 0, 0, 0, 0, 1, 1, 2, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 2, 0, 0, 0, 0, 0, 0, 1], dtype=int32)

In [22]:
# add clustering labels
neighborhoods_venues_sorted["Cluster Labels"] = kmeans.labels_
neighborhoods_venues_sorted["Cluster Labels"].astype(int)
toronto_merged = df

# merge toronto_grouped with df to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged.dropna(inplace=True)
toronto_merged.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
0,M3A,North York,Parkwoods,43.75245,-79.32991,Food & Drink Shop,Park,Flea Market,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Donut Shop,Farm,1.0
1,M4A,North York,Victoria Village,43.73057,-79.31306,Park,Grocery Store,Falafel Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Yoga Studio,Donut Shop,1.0
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264,Coffee Shop,Breakfast Spot,Yoga Studio,Thrift / Vintage Store,Spa,Event Space,Electronics Store,Food Truck,Restaurant,Distribution Center,0.0
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042,Clothing Store,Toy / Game Store,Men's Store,Food Court,Bookstore,American Restaurant,Furniture / Home Store,Restaurant,Women's Store,Cosmetics Shop,0.0
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66253,-79.39188,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Burrito Place,Gastropub,Park,Bank,Mediterranean Restaurant,Falafel Restaurant,0.0


In [23]:
#Get Toronto geocodes
address = 'Toronto,Ontario,Canada'
geolocator =geopy.geocoders.Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [24]:
#Cluster 1
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
2,"Regent Park, Harbourfront",Coffee Shop,Breakfast Spot,Yoga Studio,Thrift / Vintage Store,Spa,Event Space,Electronics Store,Food Truck,Restaurant,Distribution Center,0.0
3,"Lawrence Manor, Lawrence Heights",Clothing Store,Toy / Game Store,Men's Store,Food Court,Bookstore,American Restaurant,Furniture / Home Store,Restaurant,Women's Store,Cosmetics Shop,0.0
4,"Queen's Park, Ontario Provincial Government",Coffee Shop,Sandwich Place,Café,Italian Restaurant,Burrito Place,Gastropub,Park,Bank,Mediterranean Restaurant,Falafel Restaurant,0.0
5,"Islington Avenue, Humber Valley Village",Pharmacy,Park,Café,Shopping Mall,Bank,Skating Rink,Grocery Store,Event Space,Falafel Restaurant,Farm,0.0
7,Don Mills,Burger Joint,Gym,Bubble Tea Shop,Beer Store,Supermarket,Intersection,Grocery Store,Coffee Shop,Park,Spa,0.0
8,"Parkview Hill, Woodbine Gardens",Pizza Place,Athletics & Sports,Rock Climbing Spot,Café,Fast Food Restaurant,Flea Market,Pharmacy,Breakfast Spot,Gastropub,Bank,0.0
9,"Garden District, Ryerson",Coffee Shop,Clothing Store,Middle Eastern Restaurant,Cosmetics Shop,Hotel,Japanese Restaurant,Café,Sandwich Place,Theater,Bubble Tea Shop,0.0
10,Glencairn,Grocery Store,Pizza Place,Pub,Fast Food Restaurant,Latin American Restaurant,Sushi Restaurant,Gas Station,Bakery,Japanese Restaurant,Bank,0.0
11,"West Deane Park, Princess Gardens, Martin Grov...",Pizza Place,Sandwich Place,Chinese Restaurant,Tea Room,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Donut Shop,Ethiopian Restaurant,0.0
12,"Rouge Hill, Port Union, Highland Creek",Bar,Moving Target,Yoga Studio,Falafel Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Farm,Food,0.0


In [25]:
#Cluster 2
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
0,Parkwoods,Food & Drink Shop,Park,Flea Market,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Donut Shop,Farm,1.0
1,Victoria Village,Park,Grocery Store,Falafel Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Yoga Studio,Donut Shop,1.0
27,Hillcrest Village,Park,Bus Stop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Yoga Studio,Donut Shop,1.0
35,"East Toronto, Broadview North (Old East York)",Park,Intersection,Falafel Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Yoga Studio,Donut Shop,1.0
45,"York Mills, Silver Hills",Park,Farm,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Yoga Studio,Donut Shop,1.0
69,"High Park, The Junction South",Park,Farm,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Yoga Studio,Donut Shop,1.0
91,Rosedale,Park,Playground,Bike Trail,Yoga Studio,Falafel Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,1.0
98,"The Kingsway, Montgomery Road, Old Mill North",Park,Lounge,Farm,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Yoga Studio,1.0


In [26]:
#Cluster 3
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
50,Humber Summit,Construction & Landscaping,Auto Garage,Falafel Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Farm,Donut Shop,2.0
71,"Wexford, Maryvale",Convenience Store,Auto Garage,Yoga Studio,Farm,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,2.0


In [27]:
#Cluster 4
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
28,"Bathurst Manor, Wilson Heights, Downsview North",Business Service,Yoga Studio,Dry Cleaner,Flea Market,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,3.0
68,"Forest Hill North & West, Forest Hill Road Park",Park,Business Service,Falafel Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Yoga Studio,Donut Shop,3.0


In [28]:
#Cluster 5
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
6,"Malvern, Rouge",Fast Food Restaurant,Yoga Studio,Donut Shop,Flea Market,Fish Market,Fish & Chips Shop,Field,Farmers Market,Farm,Falafel Restaurant,4.0
